<a href="https://colab.research.google.com/github/GuysBarash/Genetic-programing-with-DEAP/blob/master/evolutionarty_GP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [100]:
!apt install libgraphviz-dev
!pip install pygraphviz
import pygraphviz as pgv

!pip install DEAP

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libgraphviz-dev is already the newest version (2.40.1-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 43 not upgraded.


In [0]:
import random
import operator
import os
import numpy as np
import pandas as pd

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp
from datetime import datetime
from deap.tools import History

from IPython.display import display
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score, recall_score, precision_score, fbeta_score
from sklearn import preprocessing

# Generate Data

In [0]:
import sys
colab_mode = 'google.colab' in sys.modules

Fetch data

In [103]:
if not colab_mode:
  train_set = r"C:\school\evolutionary\ex2\validate.csv" # This is where the file is kept
  rawdatadf = pd.read_csv(train_set, header=None)
else:
  from google.colab import drive

  drive.mount('/content/drive')
  base_folder = r'/content/drive/My Drive/colab_storage'
  train_set = os.path.join(base_folder ,'validate.csv')
  rawdatadf = pd.read_csv(train_set, header=None)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Normalize data

In [0]:
datacols = rawdatadf.columns[1:]

def get_last_n_tuples_from_cols(n, cols):
    return cols[-4 * n:]


datacols = rawdatadf.columns[1:]
datacols = get_last_n_tuples_from_cols(7, datacols)
datadf = pd.DataFrame(columns=datacols,
                      index=range(rawdatadf.shape[0]),
                      data=rawdatadf[datacols])

datadf -= datadf.mean(axis=0)
datadf /= datadf.std(axis=0)

normal = pd.Series(index=datadf.columns,
                   data=np.fmax(-datadf.min(axis=0), datadf.max(axis=0)))
datadf /= normal

labels = rawdatadf[rawdatadf.columns[0]]
labelcol = 'LABEL'
datadf[labelcol] = labels

In [105]:
top_n = 14
display(datadf.head(14))
print("")
print(f"Displaying top {top_n} out of {len(datadf)}")

,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,LABEL
0,-0.001402,-0.014584,-0.004399,-0.024160,-0.002077,-0.008590,-0.004185,-0.016526,-0.001529,-0.012480,-0.003596,-0.022150,-0.003091,-0.014508,-0.003448,-0.021577,-0.002843,-0.029625,-0.004603,-0.037046,-0.003401,-0.032067,-0.004950,-0.040553,-0.002354,-0.027773,-0.001771,-0.035057,1
1,-0.002213,-0.061451,0.001638,-0.069692,0.005580,-0.081389,0.007149,-0.087567,0.001934,-0.086130,0.000699,-0.096094,0.003699,-0.079504,0.004542,-0.084546,-0.000004,-0.076306,-0.002991,-0.083309,-0.006475,-0.072087,-0.008862,-0.082736,-0.002621,-0.075680,-0.001046,-0.084795,0
2,0.005095,-0.035076,0.006347,-0.036521,0.002121,-0.052431,0.002669,-0.054556,0.003878,-0.063283,0.001422,-0.070275,0.003292,-0.068101,0.004406,-0.075421,0.003952,-0.062126,-0.002043,-0.070617,-0.000806,-0.051912,-0.002289,-0.058583,0.004514,-0.049390,0.000099,-0.058649,1
3,-0.003863,-0.029186,-0.004863,-0.036023,-0.003019,-0.029522,-0.003248,-0.036856,-0.003468,-0.032950,-0.005717,-0.038620,-0.003763,-0.028559,-0.004275,-0.033892,-0.000536,-0.028205,-0.001701,-0.035019,-0.002565,-0.031197,-0.003926,-0.039681,-0.001933,-0.030462,-0.001965,-0.038189,0
4,-0.006349,0.047765,-0.006198,0.056993,-0.005019,0.048454,-0.007259,0.057186,-0.004250,0.041095,-0.007257,0.050974,-0.004934,0.050903,-0.005430,0.054538,-0.005120,0.049164,-0.007653,0.050662,-0.006681,0.034270,-0.009311,0.040950,-0.003591,0.034442,-0.002464,0.043321,0
5,-0.006279,-0.028841,-0.006165,-0.029359,-0.004779,-0.008281,-0.005863,-0.011942,-0.004051,-0.015899,-0.006830,-0.012665,-0.004783,-0.023954,-0.005374,-0.024017,-0.004668,-0.019518,-0.007209,-0.021039,-0.006232,-0.008172,-0.008853,-0.010035,-0.003423,0.006336,-0.002372,0.012754,1
6,-0.001547,0.031227,0.002238,0.024737,-0.002638,0.023561,-0.003341,0.015781,-0.002282,0.017545,-0.001941,0.010118,-0.000392,0.012378,0.000312,0.005356,-0.000198,0.012608,-0.001396,0.004281,0.000083,0.004509,0.004035,-0.001335,-0.000335,-0.005384,-0.000662,-0.012629,0
7,-0.002838,0.079310,-0.003865,0.080710,-0.003597,0.083210,-0.004852,0.081541,-0.003518,0.075197,-0.005972,0.074191,-0.004489,0.066595,-0.004991,0.063766,-0.002751,0.071799,-0.004401,0.069915,-0.004653,0.077518,-0.006209,0.078729,-0.002067,0.069337,-0.001386,0.068857,0
8,-0.004150,0.062259,-0.003870,0.057962,-0.002649,0.064576,-0.003808,0.059544,-0.002277,0.052199,-0.003874,0.047088,-0.001874,0.048319,-0.002543,0.043413,-0.003236,0.059048,-0.005856,0.055917,-0.002556,0.069770,-0.003904,0.068036,-0.001378,0.062491,-0.001110,0.059329,1
9,-0.004925,0.070658,-0.005025,0.071348,-0.003172,0.076086,-0.004511,0.076090,-0.003569,0.073296,-0.005175,0.068997,-0.003952,0.065615,-0.004195,0.063832,-0.003843,0.060735,-0.003988,0.055531,-0.005912,0.062787,-0.006670,0.064348,-0.003215,0.066701,-0.001595,0.067490,0



Displaying top 14 out of 50000


Check with SVM as baseline

In [0]:
# Solve with SVM
from sklearn import svm

# print("Calculating SVM")
# clf = svm.LinearSVC()
# clf.fit(datadf[datacols], datadf[labelcol])
# predictions = clf.predict(datadf[datacols])
# true_results = datadf[labelcol]
# scoring_sr = pd.DataFrame(dtype=np.float, columns=['value'])
# scoring_sr.loc['Accuracy', 'value'] = accuracy_score(true_results, predictions)
# scoring_sr.loc['Recall', 'value'] = recall_score(true_results, predictions)
# scoring_sr.loc['precision', 'value'] = precision_score(true_results, predictions)
# scoring_sr.loc['F0.25', 'value'] = fbeta_score(true_results, predictions, beta=0.25)

# display(scoring_sr)
# print("DONE WITH SVM")

# Defining evolution parameters

register operands

In [0]:
def activation(a):
    return np.tanh(a)


def neg(a):
    return -a


def double(a):
    return 2.0 * a


def half(a):
    return 0.5 * a

In [0]:
pset = gp.PrimitiveSet("MAIN", 5, "IN")
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(np.fmax, 2)
pset.addPrimitive(np.square, 1)
pset.addPrimitive(neg, 1)
pset.addPrimitive(double, 1)
pset.addPrimitive(half, 1)
pset.addPrimitive(activation, 1)
pset.addTerminal(np.float64(1.0))
pset.addTerminal(np.float64(0.5))
pset.addTerminal(np.float64(2.0))

Support functions

In [0]:
def individual_to_function(individual):
    return toolbox.compile(expr=individual)


def apply_function_to_df(func, dataset=None):
    if dataset is None:
        dataset = datadf[datacols]

    residual = np.zeros(datadf[datacols].shape[0])
    for pos in np.arange(0, datadf[datacols].shape[1], step=2):
        if pos + 3 >= datadf[datacols].shape[1]:
          break
        col1, col2, col3, col4 = datacols[pos], datacols[pos + 1], datacols[pos + 2], datacols[pos + 3]
        residual = func(residual, datadf[col1], datadf[col2], datadf[col3], datadf[col4])
    if np.isscalar(residual):
        residual = np.full(datadf[datacols].shape[0], residual)

    results = (residual > 0.0).astype(int)
    return results


def fitness_function(individual):
    global evaluation_counter
    evaluation_counter += 1
    func = individual_to_function(individual)
    results = apply_function_to_df(func)

    # Fitness is accuracy
    # fitness_score = accuracy_score(datadf[labelcol], results)
    # Fitness is F0.25 score
    fitness_score = fbeta_score(datadf[labelcol], results, beta=0.25)
    return fitness_score,

In [0]:
calculations_time = datetime.now()


def calc_time(*args):
    global calculations_time
    now = datetime.now()
    ret = now - calculations_time
    calculations_time = now
    return ret

Register hyper parameters

In [0]:
# random.seed(10)
population_size = 400
number_of_generations = 450

In [112]:
history = History()

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genFull, pset=pset, min_=2, max_=8)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)
toolbox.register("evaluate", fitness_function)
toolbox.register("select", tools.selTournament, tournsize=10)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genGrow, min_=0, max_=8)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

# Bloat control
toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))

pop = toolbox.population(n=population_size)
hof = tools.HallOfFame(1)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)
stats.register("TIME", calc_time)

# Decorate the variation operators
toolbox.decorate("mate", history.decorator)
toolbox.decorate("mutate", history.decorator)
history.update(pop)

/usr/local/lib/python3.6/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/usr/local/lib/python3.6/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


# Evolve

In [0]:
print("Starting evolution.")
evaluation_counter = 0
start_time = datetime.now()
calculations_time = datetime.now()
_, logbook = algorithms.eaSimple(pop, toolbox, cxpb=0.8, mutpb=0.15, ngen=number_of_generations, stats=stats,
                                 halloffame=hof)


Starting evolution.


/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:679: RuntimeWarning: overflow encountered in square
  result = getattr(ufunc, method)(*inputs, **kwargs)
<string>:1: RuntimeWarning: overflow encountered in multiply


gen	nevals	avg     	std    	min	max     	TIME          
0  	400   	0.382452	0.21733	0  	0.546952	0:00:56.137213


<string>:1: RuntimeWarning: overflow encountered in square


1  	342   	0.473429	0.133755	0  	0.552136	0:00:48.728039
2  	327   	0.470183	0.140804	0  	0.552136	0:00:49.876931
3  	322   	0.464703	0.152238	0  	0.563682	0:00:37.566280
4  	341   	0.447588	0.172375	0  	0.563682	0:00:40.883680
5  	333   	0.469956	0.152235	0  	0.564161	0:00:42.963403
6  	348   	0.472281	0.148796	0  	0.573807	0:00:46.774007
7  	346   	0.493135	0.122744	0  	0.574083	0:00:51.453106
8  	323   	0.497842	0.130865	0  	0.574519	0:00:54.469844
9  	335   	0.512971	0.10891 	0  	0.575384	0:00:59.416092
10 	341   	0.507166	0.120112	0  	0.575606	0:01:04.786605
11 	313   	0.522163	0.103958	0  	0.575606	0:00:59.128342
12 	327   	0.521402	0.105433	0  	0.577253	0:01:07.216336
13 	349   	0.51944 	0.120213	0  	0.577274	0:01:24.003293
14 	320   	0.524698	0.108098	0  	0.577821	0:01:18.491992
15 	337   	0.521156	0.113321	0  	0.577821	0:01:25.006635
16 	325   	0.530888	0.0997583	0.00135588	0.57792 	0:01:23.661504
17 	334   	0.529582	0.101951 	0         	0.578026	0:01:22.611612
18 	338   	0.52

# Results

In [0]:
record = stats.compile(pop)
duration = datetime.now() - start_time
print("Run concluded.")
print("Evaluations commited: {}".format(evaluation_counter))
print("Run time: {}".format(duration))
winner_creature = hof.items[0]
winner_function = individual_to_function(winner_creature)

In [0]:
nodes, edges, labels = gp.graph(winner_creature)

In [0]:
import matplotlib.pyplot as plt
import networkx as nx
from networkx.drawing.nx_agraph import graphviz_layout

g = nx.Graph()
g.add_nodes_from(nodes)
g.add_edges_from(edges)
pos = graphviz_layout(g, prog="dot")


nx.draw_networkx_nodes(g, pos, node_size=1600)
nx.draw_networkx_edges(g, pos)
nx.draw_networkx_labels(g, pos, labels, font_size=25)
plt.rcParams["figure.figsize"] = (50, 40)
plt.show()

analyze and display

In [0]:
generations_idx = logbook.select('gen')
generations_avg = logbook.select('avg')
generations_std = logbook.select('std')
generations_max = logbook.select('max')
generations_min = logbook.select('min')

plt.figure(figsize=(20, 10))
plt.errorbar(generations_idx, generations_avg, yerr=generations_std, fmt='-o', label='AVG')
# plt.plot(generations_idx,generations_avg, '-o',label='AVG')
plt.plot(generations_max, '-o', label='BEST', color='black')
plt.plot(generations_min, '-o', label='WORST', color='red')
plt.grid()
plt.legend()
plt.xlabel('Generations')
plt.ylabel('hits')
plt.ylim(0, 1)
_ = plt.plot()

In [0]:
predictions = apply_function_to_df(winner_function).astype(np.int64)
true_results = datadf[labelcol]

scoring_sr = pd.DataFrame(dtype=np.float, columns=['value'])
scoring_sr.loc['Accuracy', 'value'] = accuracy_score(true_results, predictions)
scoring_sr.loc['Recall', 'value'] = recall_score(true_results, predictions)
scoring_sr.loc['precision', 'value'] = precision_score(true_results, predictions)
scoring_sr.loc['F0.25', 'value'] = fbeta_score(true_results, predictions, beta=0.25)

display(scoring_sr)